# Install Libraries

In [ ]:
!pip install datasets

In [ ]:
!pip install finrl

In [ ]:
!pip install stable_baselines3

In [ ]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

In [ ]:
!pip install pandas_market_calendars

In [ ]:
!git clone https://github.com/benstaf/FinRL_DeepSeek.git

In [ ]:
!git clone https://github.com/benstaf/spinningup_pytorch.git

In [ ]:
!pip install torch torchvision torchaudio
!pip install numpy scipy gymnasium pandas matplotlib seaborn
!pip install mpi4py tqdm cloudpickle

In [ ]:
!apt-get update
!apt-get install -y swig cmake build-essential
!pip install --upgrade pip setuptools wheel

In [ ]:
!pip install box2d box2d-py

In [ ]:
!pip install "gym==0.15.7"

In [ ]:
!pip install -e .

# Import Libraries


In [ ]:
%cd /content/spinningup_pytorch

In [ ]:
import sys
sys.path.append("/content/FinRL_DeepSeek/")  # Adjust path to root folder
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
#from finrl.main import check_and_make_directories
#from env_stocktrading import StockTradingEnv
import os

def check_and_make_directories(directories):
    for directory in directories:
        os.makedirs("./" + directory, exist_ok=True)  # This prevents FileEx>


check_and_make_directories([TRAINED_MODEL_DIR])
sys.path.append("/content/spinningup_pytorch")

In [ ]:
#!/usr/bin/env python
# coding: utf-8

from datasets import load_dataset
import pandas as pd
#from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories
from env_stocktrading_llm_01 import StockTradingEnv

check_and_make_directories([TRAINED_MODEL_DIR])


import numpy as np
import scipy.signal
from gymnasium.spaces import Box, Discrete

import torch
import torch.nn as nn
from torch.distributions.normal import Normal
from torch.distributions.categorical import Categorical


###############

In [ ]:
import numpy as np
import torch
from torch.optim import Adam
import gymnasium as gym
import time
import spinup.algos.pytorch.ppo.core as core
from spinup.utils.logx import EpochLogger
from spinup.utils.mpi_pytorch import setup_pytorch_for_mpi, sync_params, mpi_avg_grads
from spinup.utils.mpi_tools import mpi_fork, mpi_avg, proc_id, mpi_statistics_scalar, num_procs


# Code

In [ ]:
#train = pd.read_csv('train_data2.csv')
#dataset = load_dataset("benstaf/train_data_qwen_sentiment", data_files="train_data_qwen_sentiment.csv")
dataset = load_dataset("benstaf/nasdaq_2013_2023", data_files="train_data_deepseek_sentiment_2013_2018.csv")

# Convert to pandas DataFrame
train = pd.DataFrame(dataset['train'])

#train = pd.read_csv('train_data_qwen_sentiment.csv')

train = train.drop('Unnamed: 0',axis=1)

# Create a new index based on unique dates
unique_dates = train['date'].unique()
date_to_idx = {date: idx for idx, date in enumerate(unique_dates)}

# Create new index based on the date mapping
train['new_idx'] = train['date'].map(date_to_idx)

# Set this as the index
train = train.set_index('new_idx')


#missing values with 0
train['llm_sentiment'].fillna(0, inplace=True)


# If you are not using the data generated from part 1 of this tutorial, make sure
# it has the columns and index in the form that could be make into the environment.
# Then you can comment and skip the following two lines.
#train = train.set_index(train.columns[0])
#train.index.names = ['']


# ## Construct the environment

# Calculate and specify the parameters we need for constructing the environment.

# In[16]:


In [ ]:
train

In [ ]:
train.dtypes.reset_index().rename(columns={"index": "Column", 0: "Data Type"})


In [ ]:
print(train['date'].dtype)


In [ ]:
print(train['date'][0].dtype)


In [ ]:
print(train['tic'][0])


In [ ]:
print(type(train['llm_sentiment']))

In [ ]:
# List of desired tickers
# tickers = ['AMD', 'ADBE', 'PYPL', 'CSCO', 'NFLX', 'NVDA', 'EBAY', 'MNST', 'GOOGL']

# # Filter and sort
# train = train[train['tic'].isin(tickers)].sort_values(by='date')

# Ticker list


# Step 2: Filter by tickers and date range
tickers = ['AMD', 'ADBE', 'PYPL', 'CSCO', 'NFLX', 'NVDA', 'EBAY', 'MNST', 'GOOGL']
train = train[
    (train['tic'].isin(tickers)) &
    (train['date'] >= '2018-06-14') &
    (train['date'] <= '2018-12-28')
].sort_values(by='date')

# Step 3: Create a mapping from unique dates to a new index
date_to_idx = {date: idx for idx, date in enumerate(sorted(train['date'].unique()))}

# Step 4: Map that to a new column
train['new_idx'] = train['date'].map(date_to_idx)

# Step 5: Set new_idx as the actual DataFrame index
train.set_index('new_idx', inplace=True)

In [ ]:
# Drop the 'new_idx' column (not the index!)
if 'new_idx' in train.columns:
    train.drop(columns='new_idx', inplace=True)

In [ ]:
train.tail(100)


In [ ]:
train.shape

In [ ]:
train.to_csv('/content/working_train_ppo_llm_their_dataset.csv')

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + (1+ len(INDICATORS))*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

In [ ]:
type(INDICATORS)

In [ ]:
train.head()

In [ ]:

# In[16]:


buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)


# ## Environment for training

# In[17]:


env_train, _ = e_train_gym.get_sb_env()
#print(type(env_train))


# Initialize the environment
#env_train, _ = e_train_gym.get_sb_env()

# Check the observation space
print("Observation space:", env_train.observation_space)
print("Observation space shape:", env_train.observation_space.shape)

# Reset the environment and check the observation returned
obs = env_train.reset()
print("Observation returned from reset:", obs.shape)



# # Part 3: Train DRL Agents
# * Here, the DRL algorithms are from **[Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/)**. It's a library that implemented popular DRL algorithms using pytorch, succeeding to its old version: Stable Baselines.
# * Users are also encouraged to try **[ElegantRL](https://github.com/AI4Finance-Foundation/ElegantRL)** and **[Ray RLlib](https://github.com/ray-project/ray)**.

# In[18]:

#Custom PPO agent


In [ ]:
def combined_shape(length, shape=None):
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)


def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


def count_vars(module):
    return sum([np.prod(p.shape) for p in module.parameters()])


def discount_cumsum(x, discount):
    """
    magic from rllab for computing discounted cumulative sums of vectors.

    input:
        vector x,
        [x0,
         x1,
         x2]

    output:
        [x0 + discount * x1 + discount^2 * x2,
         x1 + discount * x2,
         x2]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]


In [ ]:
class Actor(nn.Module):

    def _distribution(self, obs):
        raise NotImplementedError

    def _log_prob_from_distribution(self, pi, act):
        raise NotImplementedError

    def forward(self, obs, act=None):
        # Produce action distributions for given observations, and
        # optionally compute the log likelihood of given actions under
        # those distributions.
        pi = self._distribution(obs)
        logp_a = None
        if act is not None:
            logp_a = self._log_prob_from_distribution(pi, act)
        return pi, logp_a

In [ ]:

class MLPCategoricalActor(Actor):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.logits_net = mlp([obs_dim] + list(hidden_sizes) + [act_dim], activation)

    def _distribution(self, obs):
        logits = self.logits_net(obs)
        return Categorical(logits=logits)

    def _log_prob_from_distribution(self, pi, act):
        return pi.log_prob(act)

In [ ]:
class MLPGaussianActor(Actor):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        log_std = -0.5 * np.ones(act_dim, dtype=np.float32)
        self.log_std = torch.nn.Parameter(torch.as_tensor(log_std))
        self.mu_net = mlp([obs_dim] + list(hidden_sizes) + [act_dim], activation)

    def _distribution(self, obs):
        mu = self.mu_net(obs)
        std = torch.exp(self.log_std)
        return Normal(mu, std)

    def _log_prob_from_distribution(self, pi, act):
        return pi.log_prob(act).sum(axis=-1)    # Last axis sum needed for Torch Normal distribution

In [ ]:
class MLPCritic(nn.Module):

    def __init__(self, obs_dim, hidden_sizes, activation):
        super().__init__()
        self.v_net = mlp([obs_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs):
        return torch.squeeze(self.v_net(obs), -1) # Critical to ensure v has right shape.

In [ ]:

class MLPActorCritic(nn.Module):
    def __init__(self, observation_space, action_space,
                 hidden_sizes=(64, 64), activation=nn.Tanh):
        super().__init__()

        obs_dim = observation_space.shape[0]

        # policy builder depends on action space
        if isinstance(action_space, Box):
            self.pi = MLPGaussianActor(obs_dim, action_space.shape[0], hidden_sizes, activation)
        elif isinstance(action_space, Discrete):
            self.pi = MLPCategoricalActor(obs_dim, action_space.n, hidden_sizes, activation)

        # build value function
        self.v = MLPCritic(obs_dim, hidden_sizes, activation)

    def step(self, obs):
        with torch.no_grad():
            pi = self.pi._distribution(obs)
            a = pi.sample()
            logp_a = self.pi._log_prob_from_distribution(pi, a)
            v = self.v(obs)
        return a.numpy(), v.numpy(), logp_a.numpy()

    def act(self, obs):
        return self.step(obs)[0]


###############################

In [ ]:
class PPOBuffer:
    """
    A buffer for storing trajectories experienced by a PPO agent interacting
    with the environment, and using Generalized Advantage Estimation (GAE-Lambda)
    for calculating the advantages of state-action pairs.
    """

    def __init__(self, obs_dim, act_dim, size, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew.item()
        self.val_buf[self.ptr] = val.item()
        self.logp_buf[self.ptr] = logp.item()
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.

        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """

        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)

        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = core.discount_cumsum(deltas, self.gamma * self.lam)

        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = core.discount_cumsum(rews, self.gamma)[:-1]

        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick
        adv_mean, adv_std = mpi_statistics_scalar(self.adv_buf)
        self.adv_buf = (self.adv_buf - adv_mean) / adv_std
        data = dict(obs=self.obs_buf, act=self.act_buf, ret=self.ret_buf,
                    adv=self.adv_buf, logp=self.logp_buf)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in data.items()}


#End definition class PPOBuffer

In [ ]:


def ppo(
    env_fn,
    actor_critic=MLPActorCritic,
    ac_kwargs=dict(hidden_sizes=[256, 128], activation=torch.nn.ReLU),
    seed=42,
    steps_per_epoch=8000,  # Larger batch size for better gradient estimation
    epochs=50,  # More epochs for convergence in a volatile environment
    gamma=0.995,  # Higher discount factor to account for long-term rewards
    clip_ratio=0.7,  # it's 90% clipping when Reduced clip ratio for stable updates
    pi_lr=3e-5,  # Lower policy learning rate
    vf_lr=1e-4,  # Lower value function learning rate
    train_pi_iters=100,  # Increased policy training iterations
    train_v_iters=100,  # Increased value function training iterations
    lam=0.95,  # GAE smoothing factor for advantage estimation
    max_ep_len=1000,  # Typical trading day or customizable period
    target_kl=0.35,  # relaxed KL divergence limit
    logger_kwargs=dict(),
    save_freq=5  # Save checkpoints more frequently
):

#OLD PPO hyperparameters:

#def ppo(env_fn, actor_critic=MLPActorCritic, ac_kwargs=dict(), seed=0,
 #       steps_per_epoch=4000, epochs=50, gamma=0.99, clip_ratio=0.2, pi_lr=3e-4,
  #      vf_lr=1e-3, train_pi_iters=80, train_v_iters=80, lam=0.97, max_ep_len=1000,
   #     target_kl=0.01, logger_kwargs=dict(), save_freq=10):
    """
    Proximal Policy Optimization (by clipping),

    with early stopping based on approximate KL

    Args:
        env_fn : A function which creates a copy of the environment.
            The environment must satisfy the OpenAI Gym API.

        actor_critic: The constructor method for a PyTorch Module with a
            ``step`` method, an ``act`` method, a ``pi`` module, and a ``v``
            module. The ``step`` method should accept a batch of observations
            and return:

            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``a``        (batch, act_dim)  | Numpy array of actions for each
                                           | observation.
            ``v``        (batch,)          | Numpy array of value estimates
                                           | for the provided observations.
            ``logp_a``   (batch,)          | Numpy array of log probs for the
                                           | actions in ``a``.
            ===========  ================  ======================================

            The ``act`` method behaves the same as ``step`` but only returns ``a``.

            The ``pi`` module's forward call should accept a batch of
            observations and optionally a batch of actions, and return:

            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``pi``       N/A               | Torch Distribution object, containing
                                           | a batch of distributions describing
                                           | the policy for the provided observations.
            ``logp_a``   (batch,)          | Optional (only returned if batch of
                                           | actions is given). Tensor containing
                                           | the log probability, according to
                                           | the policy, of the provided actions.
                                           | If actions not given, will contain
                                           | ``None``.
            ===========  ================  ======================================

            The ``v`` module's forward call should accept a batch of observations
            and return:

            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``v``        (batch,)          | Tensor containing the value estimates
                                           | for the provided observations. (Critical:
                                           | make sure to flatten this!)
            ===========  ================  ======================================


        ac_kwargs (dict): Any kwargs appropriate for the ActorCritic object
            you provided to PPO.

        seed (int): Seed for random number generators.

        steps_per_epoch (int): Number of steps of interaction (state-action pairs)
            for the agent and the environment in each epoch.

        epochs (int): Number of epochs of interaction (equivalent to
            number of policy updates) to perform.

        gamma (float): Discount factor. (Always between 0 and 1.)

        clip_ratio (float): Hyperparameter for clipping in the policy objective.
            Roughly: how far can the new policy go from the old policy while
            still profiting (improving the objective function)? The new policy
            can still go farther than the clip_ratio says, but it doesn't help
            on the objective anymore. (Usually small, 0.1 to 0.3.) Typically
            denoted by :math:`\epsilon`.

        pi_lr (float): Learning rate for policy optimizer.

        vf_lr (float): Learning rate for value function optimizer.

        train_pi_iters (int): Maximum number of gradient descent steps to take
            on policy loss per epoch. (Early stopping may cause optimizer
            to take fewer than this.)

        train_v_iters (int): Number of gradient descent steps to take on
            value function per epoch.

        lam (float): Lambda for GAE-Lambda. (Always between 0 and 1,
            close to 1.)

        max_ep_len (int): Maximum length of trajectory / episode / rollout.

        target_kl (float): Roughly what KL divergence we think is appropriate
            between new and old policies after an update. This will get used
            for early stopping. (Usually small, 0.01 or 0.05.)

        logger_kwargs (dict): Keyword args for EpochLogger.

        save_freq (int): How often (in terms of gap between epochs) to save
            the current policy and value function.

    """

    # Special function to avoid certain slowdowns from PyTorch + MPI combo.
    setup_pytorch_for_mpi()

    # Set up logger and save configuration
    logger = EpochLogger(**logger_kwargs)
    logger.save_config(locals())

    # Random seed
    seed += 10000 * proc_id()
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Instantiate environment
    env = env_fn()
    obs_dim = env.observation_space.shape
    act_dim = env.action_space.shape


    print("obs dim " + str(obs_dim))
    print("act dim " + str(act_dim))

    # Create actor-critic module
    ac = actor_critic(env.observation_space, env.action_space, **ac_kwargs)

    # Sync params across processes
    sync_params(ac)

    # Count variables
    var_counts = tuple(core.count_vars(module) for module in [ac.pi, ac.v])
    logger.log('\nNumber of parameters: \t pi: %d, \t v: %d\n'%var_counts)

    # Set up experience buffer
    local_steps_per_epoch = int(steps_per_epoch / num_procs())
    buf = PPOBuffer(obs_dim, act_dim, local_steps_per_epoch, gamma, lam)

    # Set up function for computing PPO policy loss
    def compute_loss_pi(data):
        obs, act, adv, logp_old = data['obs'], data['act'], data['adv'], data['logp']

        # Policy loss
        pi, logp = ac.pi(obs, act)
        ratio = torch.exp(logp - logp_old)
        clip_adv = torch.clamp(ratio, 1-clip_ratio, 1+clip_ratio) * adv
        loss_pi = -(torch.min(ratio * adv, clip_adv)).mean()

        # Useful extra info
        approx_kl = (logp_old - logp).mean().item()
        ent = pi.entropy().mean().item()
        clipped = ratio.gt(1+clip_ratio) | ratio.lt(1-clip_ratio)
        clipfrac = torch.as_tensor(clipped, dtype=torch.float32).mean().item()
        pi_info = dict(kl=approx_kl, ent=ent, cf=clipfrac)

        return loss_pi, pi_info

    # Set up function for computing value loss
    def compute_loss_v(data):
        obs, ret = data['obs'], data['ret']
        return ((ac.v(obs) - ret)**2).mean()

    # Set up optimizers for policy and value function
    pi_optimizer = Adam(ac.pi.parameters(), lr=pi_lr)
    vf_optimizer = Adam(ac.v.parameters(), lr=vf_lr)

    # Set up model saving
    logger.setup_pytorch_saver(ac)

    def update():
        data = buf.get()

        pi_l_old, pi_info_old = compute_loss_pi(data)
        pi_l_old = pi_l_old.item()
        v_l_old = compute_loss_v(data).item()

        # Train policy with multiple steps of gradient descent
        for i in range(train_pi_iters):
            pi_optimizer.zero_grad()
            loss_pi, pi_info = compute_loss_pi(data)
            kl = mpi_avg(pi_info['kl'])
            if kl > 1.5 * target_kl:
                logger.log('Early stopping at step %d due to reaching max kl.'%i)
                break
            loss_pi.backward()
            mpi_avg_grads(ac.pi)    # average grads across MPI processes
            pi_optimizer.step()

        logger.store(StopIter=i)

        # Value function learning
        for i in range(train_v_iters):
            vf_optimizer.zero_grad()
            loss_v = compute_loss_v(data)
            loss_v.backward()
            mpi_avg_grads(ac.v)    # average grads across MPI processes
            vf_optimizer.step()

        # Log changes from update
        kl, ent, cf = pi_info['kl'], pi_info_old['ent'], pi_info['cf']
        logger.store(LossPi=pi_l_old, LossV=v_l_old,
                     KL=kl, Entropy=ent, ClipFrac=cf,
                     DeltaLossPi=(loss_pi.item() - pi_l_old),
                     DeltaLossV=(loss_v.item() - v_l_old))

    # Prepare for interaction with environment
    start_time = time.time()
    o, ep_ret, ep_len = env.reset(), 0, 0

    # Main loop: collect experience in env and update/log each epoch
    for epoch in range(epochs):
        for t in range(local_steps_per_epoch):
            a, v, logp = ac.step(torch.as_tensor(o, dtype=torch.float32))
            #print("🔥 PPO received state:", o.shape)

            next_o, r, d, _ = env.step(a)
            ep_ret += r
            ep_len += 1

            # save and log
            buf.store(o, a, r, v, logp)
            logger.store(VVals=v)

            # Update obs (critical!)
            o = next_o

            timeout = ep_len == max_ep_len
            terminal = d or timeout
            epoch_ended = t==local_steps_per_epoch-1

            if terminal or epoch_ended:
                if epoch_ended and not(terminal):
                    print('Warning: trajectory cut off by epoch at %d steps.'%ep_len, flush=True)
                # if trajectory didn't reach terminal state, bootstrap value target
                if timeout or epoch_ended:
                    _, v, _ = ac.step(torch.as_tensor(o, dtype=torch.float32))
                else:
                    v = 0
                buf.finish_path(v)
                if terminal:
                    # only save EpRet / EpLen if trajectory finished
                    logger.store(EpRet=ep_ret, EpLen=ep_len)
                o, ep_ret, ep_len = env.reset(), 0, 0


        # Save model
        if (epoch % save_freq == 0) or (epoch == epochs-1):
            logger.save_state({'env': env}, None)

        # Perform PPO update!
        update()

        # Log info about epoch
        logger.log_tabular('Epoch', epoch)
        logger.log_tabular('EpRet', with_min_and_max=True)
        logger.log_tabular('EpLen', average_only=True)
        logger.log_tabular('VVals', with_min_and_max=True)
        logger.log_tabular('TotalEnvInteracts', (epoch+1)*steps_per_epoch)
        logger.log_tabular('LossPi', average_only=True)
        logger.log_tabular('LossV', average_only=True)
        logger.log_tabular('DeltaLossPi', average_only=True)
        logger.log_tabular('DeltaLossV', average_only=True)
        logger.log_tabular('Entropy', average_only=True)
        logger.log_tabular('KL', average_only=True)
        logger.log_tabular('ClipFrac', average_only=True)
        logger.log_tabular('StopIter', average_only=True)
        logger.log_tabular('Time', time.time()-start_time)
        logger.dump_tabular()
    return ac


###################################################

In [ ]:

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--env', type=str, default=env_train) #'HalfCheetah-v2')


In [ ]:
#parser.add_argument('--hid', type=int, default=64)
#parser.add_argument('--l', type=int, default=2)
#parser.add_argument('--gamma', type=float, default=0.99)
#parser.add_argument('--seed', '-s', type=int, default=0)
#parser.add_argument('--cpu', type=int, default=4)
#parser.add_argument('--steps', type=int, default=4000)
#parser.add_argument('--epochs', type=int, default=50)
#parser.add_argument('--exp_name', type=str, default='ppo')



# Parser arguments adapted to hyperparameter optimization by chatGPT

parser.add_argument('--hid', type=int, default=512)  # Updated to match the first hidden size in ac_kwargs
parser.add_argument('--l', type=int, default=2)  # Updated to match the number of layers in ac_kwargs
parser.add_argument('--gamma', type=float, default=0.995)  # Updated to match gamma in ppo
parser.add_argument('--seed', '-s', type=int, default=42)  # Updated to match seed in ppo
parser.add_argument('--cpu', type=int, default=4)  # Kept as is since it's not in ppo
parser.add_argument('--steps', type=int, default=8000) #8000)  # Updated to match steps_per_epoch in ppo
parser.add_argument('--epochs', type=int, default=50) # 10 for test otherwise 100 it is too much 150)  # Updated to match epochs in ppo
parser.add_argument('--exp_name', type=str, default='ppo')  # Kept as is since it's not in ppo





In [ ]:
#parser.add_argument('-f', '--file', type=str, help='Kernel connection file')  # Add this line
parser.add_argument('extra_args', nargs=argparse.REMAINDER)  # Catch-all for unrecognized arguments

# args = parser.parse_args()

#mpi_fork(args.cpu)  # run parallel code with mpi  doesn't work in colab

In [ ]:
# Add this line to ignore unknown args
args, unknown = parser.parse_known_args()

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/My Drive/FinRL/finrl_ppo_models"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
save_dir = "/content/drive/My Drive/FinRL/finrl_ppo_models"
os.makedirs(save_dir, exist_ok=True)

In [ ]:

from spinup.utils.run_utils import setup_logger_kwargs


logger_kwargs = setup_logger_kwargs(args.exp_name, args.seed)

trained_ppo=ppo(lambda : args.env, actor_critic=MLPActorCritic,
        ac_kwargs=dict(hidden_sizes=[args.hid]*args.l), gamma=args.gamma,
        seed=args.seed, steps_per_epoch=args.steps, epochs=args.epochs,
        logger_kwargs=logger_kwargs)



# Save the model
model_path = model_path = "/content/drive/My Drive/FinRL/finrl_ppo_models/finRL_working_subset_train_ppo_llm_50_epochs_8k_steps_01_.pth"
torch.save(trained_ppo.state_dict(), model_path)
print("Training finished and saved in " + model_path)
# Load the model
#loaded_ppo = MLPActorCritic()
#loaded_ppo.load_state_dict(torch.load(model_path))
#loaded_ppo.eval()  # Set the model to evaluation mode



#trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo")


# ## Save the trained agent
# Trained agents should have already been saved in the "trained_models" drectory after you run the code blocks above.
#
# For Colab users, the zip files should be at "./trained_models" or "/content/trained_models".
#
# For users running on your local environment, the zip files should be at "./trained_models".